In [34]:
import pandas as pd
from math import sin, cos, sqrt, atan2


In [35]:
data = pd.read_excel('bairros.xlsx')

In [36]:
data.insert(0,'label', '0')


In [37]:
for i in range(0,data.shape[0]):
    data.at[i, 'label'] = data.at[i, 'label'].replace('0',str(i))

In [38]:
data.at[0, 'label']

'0'

In [39]:

data

,label,Bairro,Latitude,Longitude
0,0,Batista Campos,-1460744,-48490488
1,1,Cidade Velha,-1461183,-48504647
2,2,Jurunas,-1470104,-48495129
3,3,Campina,-1452948,-48499645
4,4,Reduto,-1447956,-48492640


In [23]:
def distFunc(lat1, lat2, lon1, lon2):
    R = 6373.0
    dlon = int(lon2) - int(lon1)
    dlat = int(lat2) - int(lat1)
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

In [9]:
def RangeQuery(data, Q, eps):
    neighbors = []
    for p in range(0,data.shape[0]):
        if Q == p: continue
        lat1 = data.at[Q, 'Latitude']
        lat2 = data.at[p, 'Latitude']
        lon1 = data.at[Q, 'Longitude']
        lon2 = data.at[p, 'Longitude']
        dist = distFunc(lat1, lat2, lon1, lon2)
        '''print('Q',Q)
        print('p',p)
        print(lat1,lat2,lon1,lon2,dist)
        print(eps)'''
        if dist <= eps:
            neighbors += [p]
    return neighbors

In [42]:
def DBSCAN(data, eps, min_points):
    c = 0
    for p in range(0,data.shape[0]):
        n = []
        if data.at[p, 'label'] == None: continue
        n += RangeQuery(data, p, eps)
        if len(n) < min_points:
            data.at[p, 'label'] = 'Noise'
            continue
        c += 1
        data.at[p, 'label']= c
        s = []
        s = n.remove(p)
        for q in s:
            if data.at[q, 'label'] == 'Noise': data.at[q, 'label'] = c
            if data.at[q, 'label'] == None: continue
            data.at[q, 'label'] = c
            n += RangeQuery(data, q, eps)
            if len(n) >= min_points:
                s += n
                continue


<h1>Problemas</h1>
<ol>
    <li>iterar no dataframe</li>
    <li>alterar/verificar a label</li>
    <li>remover elementos repetidos</li>
</ol>

In [11]:
shape = data.shape[0]
for i in range(0,shape):
    if data.at[i, 'label'] == None: continue
    data.at[i, 'label'] = data.at[i, 'label'].replace('Noise','1')
data 

,Bairro,label,Latitude,Longitude
0,Batista Campos,1,-1460744,-48490488
1,Cidade Velha,1,-1461183,-48504647
2,Jurunas,1,-1470104,-48.495129
3,Campina,2,-1452948,-48499645
4,Reduto,1,-1447956,-48492640


In [9]:
data.at[3, 'label'] = data.at[3, 'label'].replace('Noise','2')

In [46]:
lat1 = data.at[0, 'Latitude']
lat2 = data.at[1, 'Latitude']
lon1 = data.at[0, 'Longitude']
lon2 = data.at[1, 'Longitude']
distFunc(lat1,lat2,lon1,lon2)

Result 14922.672449175263


In [40]:
DBSCAN(data,0.2,2)

[]
[]
[]
[]
[]


In [14]:
%matplotlib inline

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from numpy import random, float


model = KMeans(n_clusters=3)

# Note I'm scaling the data to normalize it! Important for good results.
model = model.fit(scale(data))

# We can look at the clusters each data point was assigned to
print(model.labels_)


[0 2 0 1 1]


In [41]:
data

,label,Bairro,Latitude,Longitude
0,Noise,Batista Campos,-1460744,-48490488
1,Noise,Cidade Velha,-1461183,-48504647
2,Noise,Jurunas,-1470104,-48495129
3,Noise,Campina,-1452948,-48499645
4,Noise,Reduto,-1447956,-48492640


In [ ]:
from sklearn.clusters import DBSCAN

clustering = DBSCAN( min_samples=2).fit(X)
clustering.labels_